# **Proyek Akhir: Membuat Model Sistem Rekomendasi Berita**

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

## **I. Pendahuluan**
Proyek ini bertujuan untuk membangun sistem rekomendasi berita menggunakan dua pendekatan utama: **Content-Based Filtering** dan **Collaborative Filtering**. Tujuannya adalah memberikan rekomendasi yang dipersonalisasi kepada pengguna berdasarkan konten berita dan riwayat interaksi mereka.

### **Menyiapkan Library yang Dibutuhkan**

In [ ]:
# Library untuk analisis dan manipulasi data
import pandas as pd
import numpy as np
from collections import defaultdict

# Library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns

# Library untuk pemodelan Content-Based Filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Library untuk pemodelan Collaborative Filtering
# Pastikan sudah menginstal: pip install scikit-surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split as surprise_train_test_split

# Mengatur opsi tampilan pandas
pd.set_option('display.max_columns', None)

print("Semua library berhasil diimpor.")

## **II. Data Understanding**
Tahap ini berfokus pada pemuatan dan pemahaman awal terhadap dataset. Kita akan menggunakan dataset MIND-small dari Kaggle.

### **Memuat Dataset**
Kita akan memuat file `news.tsv` dan `behaviors.tsv` ke dalam DataFrame pandas.

In [ ]:
# Fungsi untuk memuat data dengan penanganan error
def load_data(file_path, column_names):
    """
    Memuat data dari file TSV ke dalam DataFrame pandas.
    
    Args:
        file_path (str): Path ke file tsv.
        column_names (list): Daftar nama kolom untuk DataFrame.
    
    Returns:
        DataFrame: DataFrame yang berisi data dari file.
    """
    try:
        df = pd.read_csv(file_path, sep='\t', names=column_names)
        print(f"File '{file_path}' berhasil dimuat.")
        return df
    except FileNotFoundError:
        print(f"Error: File '{file_path}' tidak ditemukan. Pastikan path file sudah benar.")
        return None

# Path ke file dataset (sesuaikan jika perlu)
path_to_news = './MINDsmall_train/news.tsv'
path_to_behaviors = './MINDsmall_train/behaviors.tsv'

# Mendefinisikan nama kolom berdasarkan dokumentasi dataset
news_cols = ['News_ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title_Entities', 'Abstract_Entities']
behaviors_cols = ['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions']

# Memuat data
df_news = load_data(path_to_news, news_cols)
df_behaviors = load_data(path_to_behaviors, behaviors_cols)

### **Informasi Dasar Dataset**
Melihat beberapa baris pertama, informasi, dan statistik deskriptif dari data.

In [ ]:
# Menampilkan 5 baris pertama dari data berita
if df_news is not None:
    print("Contoh Data Berita (df_news):")
    display(df_news.head())

# Menampilkan 5 baris pertama dari data perilaku
if df_behaviors is not None:
    print("\nContoh Data Perilaku (df_behaviors):")
    display(df_behaviors.head())

In [ ]:
# Menampilkan informasi ringkas tentang DataFrame berita
if df_news is not None:
    print("Informasi df_news:")
    df_news.info()

# Menampilkan informasi ringkas tentang DataFrame perilaku
if df_behaviors is not None:
    print("\nInformasi df_behaviors:")
    df_behaviors.info()

### **Exploratory Data Analysis (EDA)**
Melakukan analisis data eksplorasi untuk memahami distribusi data.

#### **Distribusi Kategori Berita**

In [ ]:
if df_news is not None:
    plt.figure(figsize=(12, 6))
    sns.countplot(y='Category', data=df_news, order=df_news['Category'].value_counts().index)
    plt.title('Distribusi Kategori Berita')
    plt.xlabel('Jumlah Artikel')
    plt.ylabel('Kategori')
    plt.show()

## **III. Data Preparation**
Menyiapkan data agar siap untuk proses pemodelan.

### **Membersihkan dan Menggabungkan Data**
Kita akan membersihkan data dari nilai null dan menggabungkan informasi yang relevan.

In [ ]:
if df_news is not None and df_behaviors is not None:
    # Membersihkan data berita dari nilai null pada kolom penting
    df_news_cleaned = df_news.dropna(subset=['Title', 'Category'])
    print(f"Jumlah data berita setelah dibersihkan: {len(df_news_cleaned)}")

    # Mengambil sampel data perilaku untuk mempercepat proses (opsional, bisa di-skip jika komputasi kuat)
    df_behaviors_sample = df_behaviors.sample(n=50000, random_state=42)

    # Memproses kolom 'History' untuk mendapatkan interaksi user-item
    user_history_list = []
    for index, row in df_behaviors_sample.iterrows():
        if isinstance(row['History'], str):
            history_ids = row['History'].split()
            for news_id in history_ids:
                user_history_list.append({'User_ID': row['User_ID'], 'News_ID': news_id})

    df_interaction = pd.DataFrame(user_history_list)
    print("Contoh data interaksi dari 'History':")
    display(df_interaction.head())

    # Menggabungkan data interaksi dengan data berita
    df_full_interaction = pd.merge(df_interaction, df_news_cleaned, on='News_ID', how='inner')
    print("Contoh data interaksi yang sudah digabungkan dengan detail berita:")
    display(df_full_interaction.head())

## **IV. Modeling: Content-Based Filtering**
Membuat model yang merekomendasikan berita berdasarkan kemiripan konten.

In [ ]:
# Mengambil data berita unik untuk model content-based
df_content = df_news_cleaned.drop_duplicates(subset=['News_ID']).reset_index(drop=True)

# Membuat fitur 'content' dengan menggabungkan judul dan kategori
df_content['content'] = df_content['Title'] + ' ' + df_content['Category']

# Inisialisasi TfidfVectorizer
tfidf = TfidfVectorizer()

# Melakukan fit dan transform pada data 'content'
tfidf_matrix = tfidf.fit_transform(df_content['content'])

# Menghitung cosine similarity dari tfidf_matrix
cosine_sim = cosine_similarity(tfidf_matrix)

# Membuat Series untuk mencocokkan judul berita dengan indeksnya
indices = pd.Series(df_content.index, index=df_content['Title'])

print("TF-IDF dan Cosine Similarity Matrix berhasil dibuat.")

### **Fungsi Rekomendasi Content-Based**

In [ ]:
def get_content_based_recommendations(title, cosine_sim_matrix=cosine_sim, df=df_content, indices_map=indices):
    """
    Memberikan rekomendasi berita berdasarkan kemiripan konten.
    
    Args:
        title (str): Judul berita yang menjadi referensi.
        cosine_sim_matrix (np.ndarray): Matriks kemiripan kosinus.
        df (DataFrame): DataFrame berisi data berita.
        indices_map (pd.Series): Mapping dari judul ke indeks.
        
    Returns:
        DataFrame: Daftar 10 berita yang direkomendasikan.
    """
    try:
        # Mendapatkan indeks dari judul berita yang diberikan
        idx = indices_map[title]
        
        # Mendapatkan skor kemiripan dari semua berita terhadap berita referensi
        sim_scores = list(enumerate(cosine_sim_matrix[idx]))
        
        # Mengurutkan berita berdasarkan skor kemiripan
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Mengambil 10 berita teratas (setelah berita referensi itu sendiri)
        sim_scores = sim_scores[1:11]
        
        # Mendapatkan indeks dari berita yang direkomendasikan
        news_indices = [i[0] for i in sim_scores]
        
        # Mengembalikan DataFrame dengan berita yang direkomendasikan
        return df[['Title', 'Category']].iloc[news_indices]
    
    except KeyError:
        return f"Error: Judul '{title}' tidak ditemukan dalam dataset."

# Contoh penggunaan
news_title_example = df_content['Title'].iloc[10]
print(f"Rekomendasi untuk berita dengan judul: '{news_title_example}'")
display(get_content_based_recommendations(news_title_example))

## **V. Modeling: Collaborative Filtering**
Membuat model yang merekomendasikan berita berdasarkan interaksi pengguna.

In [ ]:
# Menyiapkan data untuk Surprise
# Menambahkan kolom 'rating' karena data kita implisit (klik = 1)
df_interaction['Rating'] = 1

# Inisialisasi Reader dari Surprise
reader = Reader(rating_scale=(1, 1))

# Memuat data ke dalam format dataset Surprise
data_surprise = Dataset.load_from_df(df_interaction[['User_ID', 'News_ID', 'Rating']], reader)

# Inisialisasi model SVD
svd = SVD(n_factors=50, n_epochs=20, random_state=42)

# Melatih model pada seluruh dataset
trainset = data_surprise.build_full_trainset()
svd.fit(trainset)

print("Model SVD (Collaborative Filtering) berhasil dilatih.")

### **Fungsi Rekomendasi Collaborative Filtering**

In [ ]:
def get_collaborative_filtering_recommendations(user_id, model=svd, df_interact=df_interaction, df_n=df_news_cleaned):
    """
    Memberikan rekomendasi berita untuk pengguna tertentu.
    
    Args:
        user_id (str): ID pengguna.
        model: Model Surprise yang sudah dilatih.
        df_interact (DataFrame): DataFrame interaksi user-item.
        df_n (DataFrame): DataFrame berisi data berita.
        
    Returns:
        DataFrame: Daftar 10 berita yang direkomendasikan.
    """
    # Mendapatkan daftar berita yang sudah dibaca oleh pengguna
    read_news = df_interact[df_interact['User_ID'] == user_id]['News_ID'].unique()
    
    # Mendapatkan daftar semua berita unik
    all_news = df_n['News_ID'].unique()
    
    # Mendapatkan daftar berita yang belum dibaca oleh pengguna
    unread_news = [news for news in all_news if news not in read_news]
    
    # Memprediksi rating untuk berita yang belum dibaca
    predictions = [model.predict(user_id, news_id) for news_id in unread_news]
    
    # Mengurutkan prediksi berdasarkan estimasi rating tertinggi
    predictions.sort(key=lambda x: x.est, reverse=True)
    
    # Mengambil 10 rekomendasi teratas
    top_n_preds = predictions[:10]
    
    # Mendapatkan ID berita dari rekomendasi
    top_n_news_ids = [pred.iid for pred in top_n_preds]
    
    # Mengembalikan detail berita dari ID yang direkomendasikan
    return df_n[df_n['News_ID'].isin(top_n_news_ids)][['Title', 'Category']]


# Contoh penggunaan
user_id_example = df_interaction['User_ID'].iloc[0]
print(f"Rekomendasi untuk pengguna dengan ID: {user_id_example}")
display(get_collaborative_filtering_recommendations(user_id_example))

## **VI. Evaluation**
Mengevaluasi model Collaborative Filtering menggunakan metrik Precision@k.

In [ ]:
def precision_at_k(predictions, k=10, threshold=0.8):
    """
    Menghitung Precision@k dari daftar prediksi.
    
    Args:
        predictions (list): Daftar objek prediksi dari Surprise.
        k (int): Jumlah item rekomendasi.
        threshold (float): Batas rating untuk dianggap relevan (karena rating kita 1, threshold bisa < 1).
        
    Returns:
        float: Nilai Precision@k.
    """
    # Membuat map dari user ke item yang benar-benar mereka sukai (ground truth)
    user_est_true = defaultdict(list)
    for uid, _, true_r, _, _ in predictions:
        user_est_true[uid].append((true_r, _))

    precisions = dict()
    for uid, user_ratings in user_est_true.items():
        # Mengurutkan rating pengguna berdasarkan prediksi
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        
        # Menghitung jumlah item relevan di antara k rekomendasi teratas
        n_rel = sum((true_r >= threshold) for (true_r, _) in user_ratings[:k])
        
        # Menghitung precision untuk pengguna ini
        precisions[uid] = n_rel / k

    # Mengembalikan rata-rata precision dari semua pengguna
    return sum(prec for prec in precisions.values()) / len(precisions)

# Membagi data menjadi train dan test set
trainset, testset = surprise_train_test_split(data_surprise, test_size=0.2, random_state=42)

# Melatih model SVD pada trainset
svd_eval = SVD(n_factors=50, n_epochs=20, random_state=42)
svd_eval.fit(trainset)

# Membuat prediksi pada testset
predictions = svd_eval.test(testset)

# Menghitung Precision@10
p_at_10 = precision_at_k(predictions, k=10)
print(f"Hasil evaluasi model Collaborative Filtering:")
print(f"Precision@10 = {p_at_10:.4f}")